In [1]:
import tensorflow as tf
print(tf.__version__)
tf.config.list_physical_devices()

2.4.1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
train_data_directory = 'D:/data science/cats and dogs/train/'

image_size = (299,299)

In [3]:
# image data preprocessing
batch_size = 32


class_mode = 'categorical' # for multi-class classification problem, use: class_mode = 'category' 


train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2 # set validation split
    ) 

train_generator = train_datagen.flow_from_directory(
    train_data_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=class_mode, # for multi-class classification problem, use 'category'
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_directory, # same directory as training data
    target_size=image_size,
    batch_size=batch_size,
    class_mode=class_mode, # for multi-class classification problem, use 'category'
    subset='validation') # set as validation data

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [4]:
# build a model
base_model = tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

base_model.trainable = False

# Create the model
model = tf.keras.Sequential()
model.add(base_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(2, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 10, 10, 2048)      20861480  
_________________________________________________________________
flatten (Flatten)            (None, 204800)            0         
_________________________________________________________________
dense (Dense)                (None, 2)                 409602    
Total params: 21,271,082
Trainable params: 409,602
Non-trainable params: 20,861,480
_________________________________________________________________


In [5]:
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(lr=0.01),
              metrics=['acc'])

In [6]:
# Callbacks
checkpoint = tf.keras.callbacks.ModelCheckpoint("Xception_dogs_vs_cats.h5", 
                             monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto', 
                             save_freq='epoch')
early = tf.keras.callbacks.EarlyStopping(monitor='val_acc', 
                      min_delta=0, 
                      patience=10, 
                      verbose=1, 
                      mode='auto')
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc',
                                            patience=1,
                                            verbose=1,
                                            factor=0.1,
                                            min_lr=0.000000001)

In [ ]:
# fit/train model

nb_epochs = 1000

history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs,
    callbacks=[checkpoint, early, learning_rate_reduction])

Epoch 1/1000
625/625 [==============================] - 503s 795ms/step - loss: 11.1374 - acc: 0.9325 - val_loss: 10.2447 - val_acc: 0.9607

Epoch 00001: val_acc improved from -inf to 0.96074, saving model to Xception_dogs_vs_cats.h5
Epoch 2/1000
625/625 [==============================] - 375s 600ms/step - loss: 10.9833 - acc: 0.9586 - val_loss: 9.7187 - val_acc: 0.9633

Epoch 00002: val_acc improved from 0.96074 to 0.96334, saving model to Xception_dogs_vs_cats.h5
Epoch 3/1000
625/625 [==============================] - 375s 600ms/step - loss: 9.3364 - acc: 0.9675 - val_loss: 15.2073 - val_acc: 0.9599

Epoch 00003: val_acc did not improve from 0.96334

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
Epoch 4/1000
625/625 [==============================] - 375s 599ms/step - loss: 8.3380 - acc: 0.9720 - val_loss: 10.3132 - val_acc: 0.9706

Epoch 00004: val_acc improved from 0.96334 to 0.97055, saving model to Xception_dogs_vs_cats.h5
Epoch 5/1000
625/625 [=

In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()